<a href="https://colab.research.google.com/github/ad451/Stackoverflow_tag_generator/blob/main/StackOverflow_Tag_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**Importing the required modules**

In [99]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import random
from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import ToktokTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
import string





**Web Scraping the current set of questions for testing the model**

In [ ]:


Questions=[] #array to store the questions

for pageNumber in range(1,401):
    response=requests.get("https://stackoverflow.com/questions",params={"tab":"newest","page":pageNumber,"pagesize":50})

    data=BeautifulSoup(response.text,'html.parser' ) #parsing the html text

    req_data=data.find(id="questions")

    new_data=req_data.find_all("h3", class_="s-post-summary--content-title") # the tag that contains the question info



    for element in new_data:
        link=element.a.attrs['href']

        response=requests.get(f"https://stackoverflow.com/{link}")  #fetching the content related to the question

        data_questionwise=BeautifulSoup(response.text,'html.parser')

        question_wise_title=data_questionwise.find("div",id="question-header").h1.a.string #title

        question_wise_desc=data_questionwise.find("div",class_="s-prose js-post-body") #description

        all_paragraphs=question_wise_desc.find_all("p")

        total_description_question_wise=""

        for para in all_paragraphs:
            total_description_question_wise+=para.text

        Final_content=question_wise_title+""+total_description_question_wise  #concatenating the title and description

        tag_question_wise=data_questionwise.find("ul",class_="ml0 list-ls-none js-post-tag-list-wrapper d-inline").li.text #tag

        Questions.append([Final_content,tag_question_wise])

    print(pageNumber)   #checking which page questions have been fetched yet

print(len(Questions))








     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 kB 3.2 MB/s eta 0:00:00


**Sql code to combine the input questions and tags table**

In [ ]:
###################################### Code for inner combine ######################################
df1 = pd.read_csv('Questions.csv', encoding='ISO-8859-1')
df2 = pd.read_csv('Tags.csv', encoding='ISO-8859-1')

# combined dataframe of questiontags
df3 = df1.set_index('Id').join(df2.set_index('Id'))
df3 = df3.reset_index()
df4 = pd.DataFrame(df3.Id.value_counts())
df4 = df4.reset_index()

# df5 is the final combined dataframe of questiontags. Randomly picked datapoints from df3
store = {}
df5 = pd.DataFrame()
n_train = 20000
for i in range(n_train):
    key = 0
    while(key==0):
        a = random.randint(df4.shape[0])
        if a in store:
            continue
        else:
            key==1
            store[a] = 1
            ind = df3[df3.Id==df4.iloc[a,0]].first_valid_index()
            df5 =  pd.concat([df5,pd.DataFrame(df3.iloc[ind]).T],ignore_index=True)
            break

###################################### Code for removing angular brackets from the "Body" column ######################################

for j in range(df5.shape[0]):
    ans=" "
    curr = 0
    flag = 0
    while(curr<len(df5['Body'][j])):
        if( df5['Body'][j][curr]=='<'):
            flag = 1
            curr = curr + 1
            continue
        if( df5['Body'][j][curr]=='>'):
            flag = 0
            curr = curr + 1
            continue
        if(flag == 1):
            curr = curr + 1
        if(flag == 0):
            ans = ans + df5['Body'][j][curr]
            curr = curr + 1
    df1['Title'][j] += ans

df6 = pd.concat([df5.iloc[:,6], df5.iloc[:,8]], axis=1)

**Machine learning part (Preprocessing and training)**

---



In [104]:
train_data=pd.read_csv("questiontags_train.csv")
test_data=pd.read_csv("questiontags_test.csv")

In [105]:
#rename the columns of the train dataset

train_data.drop(train_data.columns[0],axis=1,inplace=True)


train_data.rename(columns={"Title":"Questions","Tag":"Tags"},inplace=True)


In [ ]:

#code to include around 10000 more questions to the train dataset from the test dataset
v={}
count=0
while count!=10000:
    idx=random.randint(0,test_data.shape[0]-1)
    if idx not in v:
      train_data=train_data.append(test_data.loc[idx],ignore_index=True)
      test_data.drop(idx,inplace=True,axis=0)
      count+=1
      v[idx]=1
    else:
      continue


In [107]:
train_data['Questions']=train_data['Questions'].str.replace('</p>',' ')
train_data['Questions']=train_data['Questions'].str.replace('\n',' ')
train_data['Questions']=train_data['Questions'].str.replace('</a>',' ')
train_data['Questions']=train_data['Questions'].apply(lambda x : re.sub('(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)' , ' ' , x)) #removing any urls
train_data['Questions'] = train_data['Questions'].apply(lambda x: ' '.join([w for w in x.split() if len(w) > 3])) #removes small length words (len<3)
train_data['Questions']=train_data['Questions'].str.replace('<a href=" ">','')

train_data['Questions']=train_data['Questions'].apply(lambda x : x.lower()) #coverting to lowercase


In [157]:
test_data['Questions']=test_data['Questions'].str.replace('</p>',' ')
test_data['Questions']=test_data['Questions'].str.replace('\n',' ')
test_data['Questions']=test_data['Questions'].str.replace('</a>',' ')
test_data['Questions'] = test_data['Questions'].apply(lambda x: ' '.join([w for w in x.split() if len(w) > 3]))
test_data['Questions']=test_data['Questions'].str.replace('<a href=" ">','')
test_data['Questions']=test_data['Questions'].apply(lambda x : x.lower())
test_data['Questions']=test_data['Questions'].apply(lambda x : re.sub('(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)' , ' ' , x)) #removing any urls

In [109]:
#removing the stop words and the punctuations from test and train dataset
nltk.download('stopwords')

stop_words=set(nltk.corpus.stopwords.words('english'))
train_data['Questions']=train_data['Questions'].apply(lambda x:' '.join([w for w in x.split() if w not in stop_words]))
test_data['Questions']=test_data['Questions'].apply(lambda x:' '.join([w for w in x.split() if w not in stop_words]))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [113]:
#removing the punctuations from test and train dataset


punctuations = string.punctuation
train_data['Questions']=train_data['Questions'].apply(lambda x:' '.join([''.join([char for char in w if char not in punctuations]) for w in x.split()]))
test_data['Questions']=test_data['Questions'].apply(lambda x:' '.join([''.join([char for char in w if char not in punctuations]) for w in x.split()]))



In [123]:
test_data=test_data.reset_index()


Index(['level_0', 'index', 'Questions', 'Tags'], dtype='object')

In [132]:
test_data.drop(['level_0'],inplace=True,axis=1)

In [148]:
#dropping the rows with empty values of Question after filtering
for j in range(len(test_data['Questions'])):
  if len(test_data['Questions'][j])==0:
     test_data.drop(j,inplace=True)

for j in range(len(train_data['Questions'])):
  if len(train_data['Questions'][j])==0:
     train_data.drop(j,inplace=True)
     print("yes")

In [151]:
test_data=test_data.reset_index()
train_data=train_data.reset_index()

In [153]:
test_data.drop(['index'],inplace=True,axis=1)
train_data.drop(['index'],inplace=True,axis=1)


(9176, 2)